In [1]:
import gc
import pandas as pd
import numpy as np
import os

from glob import glob
from tqdm import tqdm_notebook, tqdm

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify
from utils import NUM_FOLDS, FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2

%matplotlib inline

In [2]:
files = sorted(glob('../feats/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

  0%|          | 0/52 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 52/52 [00:57<00:00,  1.11s/it]


In [6]:
df = df[df['date']>'2014-04-25']

# split train & test
train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']
del df
gc.collect()

22

In [7]:
train_df.groupby('id').mean()

,cat_id,demand,demand_shift_28,demand_shift_365,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,index,item_id,month,seasonality,sell_price,snap_CA,snap_TX,snap_WI,state_id,store_id,wday,wm_yr_wk,year,demand_last_year
id,,,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,2.0,0.716438,0.709589,0.642466,4.0,14.754386,3.0,1.877193,0.0,3.491015e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,0.0,0.0,3.993151,11499.887671,2014.815068,0.642466
FOODS_1_001_CA_2_validation,2.0,0.958904,0.958904,0.924658,4.0,14.754386,3.0,1.877193,0.0,3.491315e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,0.0,1.0,3.993151,11499.887671,2014.815068,0.924658
FOODS_1_001_CA_3_validation,2.0,0.994521,0.995890,0.947945,4.0,14.754386,3.0,1.877193,0.0,3.491598e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,0.0,2.0,3.993151,11499.887671,2014.815068,0.947945
FOODS_1_001_CA_4_validation,2.0,0.319178,0.313699,0.302740,4.0,14.754386,3.0,1.877193,0.0,3.491896e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,0.0,3.0,3.993151,11499.887671,2014.815068,0.302740
FOODS_1_001_TX_1_validation,2.0,0.531507,0.549315,0.438356,4.0,14.754386,3.0,1.877193,0.0,3.492195e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,1.0,4.0,3.993151,11499.887671,2014.815068,0.438356
FOODS_1_001_TX_2_validation,2.0,0.483562,0.487671,0.435616,4.0,14.754386,3.0,1.877193,0.0,3.492495e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,1.0,5.0,3.993151,11499.887671,2014.815068,0.435616
FOODS_1_001_TX_3_validation,2.0,0.432877,0.441096,0.345205,4.0,14.754386,3.0,1.877193,0.0,3.492794e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,1.0,6.0,3.993151,11499.887671,2014.815068,0.345205
FOODS_1_001_WI_1_validation,2.0,0.453425,0.453425,0.424658,4.0,14.754386,3.0,1.877193,0.0,3.493093e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,2.0,7.0,3.993151,11499.887671,2014.815068,0.424658
FOODS_1_001_WI_2_validation,2.0,0.305479,0.290411,0.294521,4.0,14.754386,3.0,1.877193,0.0,3.493392e+07,328.0,6.523288,0.00274,2.240000,0.328767,0.328767,0.328767,2.0,8.0,3.993151,11499.887671,2014.815068,0.294521


In [5]:
test_df = df[df['date']>='2016-04-25']

In [9]:
test_df

,cat_id,d,date,demand,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,id,index,item_id,month,seasonality,sell_price,snap_CA,snap_TX,snap_WI,state_id,store_id,wday,wm_yr_wk,year,demand_last_year
45942500,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_validation,45942500,2566,4,0.408304,8.380000,0,0,0,0,0,3,11613,2016,0.0
45942501,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_002_CA_1_validation,45942501,1544,4,0.408304,3.970000,0,0,0,0,0,3,11613,2016,0.0
45942502,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_003_CA_1_validation,45942502,2732,4,0.408304,2.970000,0,0,0,0,0,3,11613,2016,0.0
45942503,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_004_CA_1_validation,45942503,837,4,0.408304,4.640000,0,0,0,0,0,3,11613,2016,3.0
45942504,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_005_CA_1_validation,45942504,1657,4,0.408304,2.880000,0,0,0,0,0,3,11613,2016,0.0
45942505,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_006_CA_1_validation,45942505,2002,4,0.408304,0.960000,0,0,0,0,0,3,11613,2016,1.0
45942506,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_007_CA_1_validation,45942506,2163,4,0.408304,7.880000,0,0,0,0,0,3,11613,2016,0.0
45942507,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_008_CA_1_validation,45942507,0,4,0.408304,0.480000,0,0,0,0,0,3,11613,2016,11.0
45942508,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_009_CA_1_validation,45942508,1,4,0.408304,1.770000,0,0,0,0,0,3,11613,2016,0.0
45942509,0,d_1914,2016-04-25,0,0,NaN,NaN,NaN,NaN,HOBBIES_1_010_CA_1_validation,45942509,1085,4,0.408304,2.970000,0,0,0,0,0,3,11613,2016,2.0


In [11]:
test_df = test_df[['id','d','demand_last_year']].pivot(index='id', columns='d', values='demand_last_year').reset_index()

In [16]:
# split test1 / test2
preds1 = test_df[['id']+COLS_TEST1]
preds2 = test_df[['id']+COLS_TEST2]

# change column names
preds1.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]
preds2.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]

# replace test2 id
preds2['id']= preds2['id'].str.replace('_validation','_evaluation')

# merge
preds = preds1.append(preds2)
preds.fillna(0,inplace=True)

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
preds.to_csv("../output/submission_shift_1y.csv", index=False)

In [20]:
from utils import NUM_FOLDS, FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2

In [18]:
pred = test_df[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [23]:
preds1 = pred[['id']+COLS_TEST1]
preds2 = pred[['id']+COLS_TEST2]

In [26]:
preds1.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]
preds2.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]

In [29]:
preds2['id']= preds2['id'].str.replace('_validation','_evaluation')

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
preds1.append(preds2)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,FOODS_1_001_CA_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,FOODS_1_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,FOODS_1_001_CA_4_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,FOODS_1_001_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,FOODS_1_001_TX_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,FOODS_1_001_TX_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,FOODS_1_001_WI_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,FOODS_1_001_WI_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,FOODS_1_001_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
